In [ ]:
#Timing of bets from position dataframe
df0=tPos[tPos==0].index
df1=tPos.shift(1)
df1=df1[df1!=0].index
bets=df0.intersection(df1)
df0=tPos.iloc[1:]*tPos.iloc[:-1].values
bets=bets.union(df0[df0<0].index).sort_values()

In [ ]:
#Estimate average holding period of a strategy, given a pandas
#series of position: tPos
def getHoldingPeriod(Pos):
    HoldPeriod=pd.DataFrame(columns=['dT','w'])
    timeEntry=0
    posDiff=Pos.diff()
    timeDiff=(Pos.index-Pos.index[0])/np.timedelta64(1,'D')
    for i in range(1,Pos.shape[0]):
        if posDiff.iloc[i]*Pos.iloc[i-1]>=0: #Increased position
            if Pos.iloc[i]!=0:
                timeEntry=(timeEntry*Pos[i-1]+timeDiff[i]*posDiff.iloc[i])/Pos.iloc[i]
                #Update average entry time, weighting positions
        else:#Decreased position
            if Pos.iloc[i]*Pos.iloc[i-1]<0: #Long to short
                HoldPeriod.loc[Pos.index[i],['dT','w']]=(
                    timeDiff[i]-timeEntry,abs(Pos.iloc[i-1]))
                timeEntry=timeDiff[i] #Reset entry time
            else:#Long to Long but decreased
                HoldPeriod.loc[Pos.index[i],['dT','w']]=(timeDiff[i]-tEntry,abs(posDiff.iloc[i]))
    if HoldPeriod['w'].sum()>0:
        HoldPeriod=(HoldPeriod['dT']*HoldPeriod['w']).sum()/HoldPeriod['w'].sum()
    else:
        HoldPeriod=np.nan
    return HoldPeriod

In [ ]:
#Returns concentration
def getHHI(betRet):
    if betRet.shape[0]<=2: return np.nan
    weight=betRet/betRet.sum()
    HHI=((weight**2).sum()-1/betRet.shape[0])/(1.-1/betRet.shape[0])
    return HHI
HHIPos=getHHI(ret[ret>=0])
HHINeg=getHHI(ret[ret<0])
MFreqHHI=getHHI(ret.groupby(pd.TimeGrouper(freq='M')).count())

In [17]:
import pandas as pd
import numpy as np
import yfinance as yf
df=yf.download('NVDA','2015-01-01','2023-05-10')
df=df[['Adj Close']]
#DrawDown and Time under Water
def computeDD_TuW(series,dollars=False):
    df0=series.to_frame('pnl')
    df0['HWM']=series.expanding().max() #?
    df1=df0.groupby('HWM').min().reset_index() #?
    df1.columns=['HWM','MIN']
    df1.index=df0['HWM'].drop_duplicates(keep='first').index #Time of High water marks
    df1=df1[df1['HWM']>df1['MIN']] #High water mark followed by a Draw down
    if dollars:
        dd=df1['HWM']-df1['MIN']
    else:
        dd=1-df1['MIN']/df1['HWM']
    TuW=((df1.index[1:]-df1.index[:-1])/np.timedelta64(1,'D')).values
    TuW=pd.Series(TuW,index=df1.index[:-1])
    return dd,TuW

[*********************100%***********************]  1 of 1 completed


In [18]:
nvda_series=df['Adj Close']
dd,TuW=computeDD_TuW(nvda_series)

In [19]:
dd

Date
2015-01-02    0.049180
2015-01-23    0.072912
2015-02-10    0.006683
2015-02-17    0.011618
2015-02-24    0.011206
                ...   
2021-10-26    0.010762
2021-11-04    0.001644
2021-11-08    0.050091
2021-11-19    0.044930
2021-11-29    0.663351
Length: 113, dtype: float64

In [20]:
TuW

Date
2015-01-02    21.0
2015-01-23    18.0
2015-02-10     7.0
2015-02-17     7.0
2015-02-24     6.0
              ... 
2021-09-03    53.0
2021-10-26     9.0
2021-11-04     4.0
2021-11-08    11.0
2021-11-19    10.0
Length: 112, dtype: float64